# Book Review Sentiment Model

This notebook consists about building sentiment analysis model from Book Review comments. All datasets was scraped from Goodreads on July 2024. Goodreads was choosen because it may have good word and labeling quality. Scraped data format is line json (`.ljson`) which is a single of datum is in one line with format json. The reason I use this format is scalability issue. Scraping method is explained in `scrape.py`.

## Requirements

To run this notebook, ensure that you have installed below dependencies:
1. Tensorflow
2. Keras
3. Python
4. Numpy
5. Nltk
6. Matplotlib
7. Sklearn
8. Gensim

In [1]:
import json
import re

import keras
import tensorflow as tf
import numpy as np
import matplotlib as mt
import matplotlib.pyplot as plt
import sklearn
import nltk

import gensim
import gensim.downloader

from gensim.models import Word2Vec, FastText
from datetime import datetime
from keras import losses
from keras import optimizers
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay, confusion_matrix


2024-07-21 22:46:55.877155: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-21 22:46:55.917267: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-21 22:46:56.590664: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Below results show the version of the library that I used

In [2]:
print(tf.__version__)
print(np.version.full_version)
print(mt.__version__)
print(sklearn.__version__)
print(nltk.__version__)
print(gensim.__version__)

2.16.1
1.26.4
3.9.0
1.5.0
3.8.1
4.3.2


Below is the constant of this notebook that I used for generating model

In [3]:
LANGUAGE = "english"
WORKER_NUMBER = 16
WORD_EPOCH = 20
INPUT_SIZE = 300

LEARNING_EPOCH = 100
PATIENCE = 5

## Text Preprocessing

The first step of creating sentiment model is text preprocessing. In this step, I remove any unnecessary words, such as 'show more' and punctuations in text. I also do case folding to lowercase in this step. After text has been formatted, I stem every word using `nltk` tools. 

In [4]:
BANNED_KEYWORDS = [
    "This entire review has been hidden because of spoilers"]
corpus = []
regex = r'[^a-zA-Z0-9\- \n"\']+'

### Text Cleaning

Below code shows text cleaning process and word tokenize. We save the result at `comments_cleaned.ljson` as cache.

In [5]:
stopwords = list(nltk.corpus.stopwords.words(LANGUAGE))

In [6]:
with open("datasets/comments_cleaned.ljson", "w") as fw:
    with open("datasets/comments.ljson") as fr:
        for line in fr:
            data = json.loads(line)

            # Skip if the comment contains banned keywords
            if any(keyword in data["text"] for keyword in BANNED_KEYWORDS):
                continue

            result = []
            for sentence in nltk.sent_tokenize(data["text"]):
                # Case folding
                sentence = sentence.lower()

                # Stopword removal
                sentence = " ".join([word for word in sentence.split() if word not in stopwords])

                # Remove special chars
                sentence = re.sub(regex, '', sentence)
                sentence = sentence.replace("-", " ")
                sentence = sentence.replace("\n", " ")
                sentence = sentence.replace("\"", "")
                sentence = re.sub(r'\bhttp[a-z0-9]+\b', '', sentence)
                sentence = re.sub(r'\b(img|src)[a-z0-9]*\b', '', sentence)
                sentence = re.sub(r'\s{2,}', ' ', sentence)

                # Remove unnecessary words
                sentence = sentence.replace("- - - - show more", "")
                sentence = sentence.replace("show more", "")
                sentence = sentence.replace("show less", "")

                # Remove whitespaces
                sentence = sentence.strip()

                if len(sentence) < 10:
                    continue

                result.append(sentence)

            # Change rating to integer
            data["rating"] = int(data["rating"].split(" ")[1])
            data["text"] = result

            if len(data["text"]) == 0:
                continue

            fw.write(json.dumps(data))
            fw.write("\n")

            corpus.extend(data["text"])

Below code shows corpus enrichment that will used for word2vec datasets.

In [7]:
with open("datasets/corpus.txt", "r") as fr:
    for line in fr:
        if any(keyword in line for keyword in BANNED_KEYWORDS):
            continue

        if len(sentence) < 10:
            continue

        for sentence in nltk.sent_tokenize(line):
            # Stopword removal
            sentence = " ".join([word for word in sentence.split() if word not in stopwords])

            sentence = sentence.replace("\n", " ")
            sentence = re.sub(regex, '', line)
            sentence = sentence.strip()
            sentence = sentence.lower()

            sentence = sentence.replace("-", " ")
            sentence = sentence.replace("\"", "")
            sentence = re.sub(r'\bhttp[a-z0-9]+\b', '', sentence)
            sentence = re.sub(r'\b(img|src)[a-z0-9]*\b', '', sentence)
            sentence = re.sub(r'\s{2,}', ' ', sentence)

            sentence = sentence.replace("- - - - show more", "")
            sentence = sentence.replace("show more", "")
            sentence = sentence.replace("show less", "")

            if len(sentence) < 10:
                continue

            if len(sentence) == 0:
                continue

            corpus.append(sentence)

with open("datasets/corpus_cleaned.txt", "w") as fw:
    fw.write("\n".join(corpus))

# Cleanup
corpus = []

### Text Stemming

Below code shows stemming process on corpus and datasets. 

In [8]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer(LANGUAGE)

Below is process of stemming process on corpus.

In [9]:
with open("datasets/corpus_stemmed.txt", "w") as fw:
  with open("datasets/corpus_cleaned.txt", "r") as fr:
      for line in fr:
          words = nltk.word_tokenize(line, language=LANGUAGE)
          result = " ".join([stemmer.stem(word) for word in words])

          fw.write(result)
          fw.write("\n")

In [10]:
with open("datasets/comments_stemmed.ljson", "w") as fw:
  with open("datasets/comments_cleaned.ljson", "r") as fr:
      for line in fr:
          data = json.loads(line)
          result = []
          
          for sentence in data["text"]:
              words = nltk.word_tokenize(sentence, language=LANGUAGE)
              sentence = " ".join([stemmer.stem(word) for word in words])
              result.append(sentence)

          fw.write(json.dumps({
              "text": result,
              "rating": data["rating"]
          }))
          fw.write("\n")

In [11]:
# Cleanup
stemmer = None

## Feature Extraction

This section will explain about feature extraction process.

In [12]:
class SentenceIterator:
    """This class is used to stream all line over the file."""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        with open(self.filename, "r") as f:
            for line in f:
                yield line.split()

### Word2Vec

In this section, we will try train Word2Vec model from scratch with text8 dataset and our scraped dataset.

In [13]:
sentence = SentenceIterator("datasets/corpus_stemmed.txt")

In [14]:
word2vec_model = Word2Vec(sentence, workers=WORKER_NUMBER, vector_size=INPUT_SIZE, epochs=WORD_EPOCH)
word2vec_model.get_latest_training_loss()

0.0

In [15]:
# Save model checkpoint
word2vec_model.save("models/corpus_word2vec.model")

In [16]:
print("Number of words in the model:", len(word2vec_model.wv.index_to_key))

Number of words in the model: 17781


In [17]:
word2vec_model.wv.most_similar("love")

[('enjoy', 0.4678633213043213),
 ('like', 0.46775925159454346),
 ('and', 0.43738266825675964),
 ('the', 0.42680367827415466),
 ('realli', 0.4210198223590851),
 ('ador', 0.4179684519767761),
 ('think', 0.4178924262523651),
 ('know', 0.3722281754016876),
 ('of', 0.3656740188598633),
 ('also', 0.3622031509876251)]

## Fasttext

In this section, we will try use fasttext model.

In [18]:
sentence = SentenceIterator("datasets/corpus_stemmed.txt")

In [19]:
fasttext_model = FastText(sentence, workers=WORKER_NUMBER, vector_size=INPUT_SIZE, epochs=WORD_EPOCH)
fasttext_model.get_latest_training_loss()

0.0

In [20]:
fasttext_model.save("models/corpus_fasttext.model")

In [21]:
print("Number of words in the model:", len(fasttext_model.wv.index_to_key))

Number of words in the model: 17781


In [22]:
fasttext_model.wv.most_similar("love")

[('loveh', 0.7908258438110352),
 ('glove', 0.7520722150802612),
 ('lovey', 0.7279291152954102),
 ('loveli', 0.6851240396499634),
 ('loveabl', 0.6354579925537109),
 ('loveless', 0.5756585597991943),
 ('lovecraft', 0.5397000312805176),
 ('lovecraftian', 0.4944160580635071),
 ('wove', 0.49380677938461304),
 ('ador', 0.4839033782482147)]

In [23]:
# Cleanup
sentence = None

## Dataset
In this section, we will learn about our dataset.

In [24]:
data_cnt = 0
rating = [0,0,0,0,0]

with open("datasets/comments_stemmed.ljson", "r") as fr:
    for line in fr:
        data_cnt += 1

        data = json.loads(line)
        rating[data["rating"] - 1] += 1

print("Number of data:", data_cnt)
print("Rating distribution:", rating)

Number of data: 24523
Rating distribution: [3279, 4371, 5333, 5808, 5732]


## Labeling and Data Split

In this section, we will label `comments_stemmed` dataset. We will use two scheme of based on rating. Sentiment classification will devide into 3 class, positive (index 0), neutral (index 1), and negative (index 2). The label will be a vector with three element with range value between 0 and 1. This label is a softmax of three class that we defined before.

The dataset will be splitted into 3 parts, training with proportion 70% of data, test with proportion 15% of data, and validation with proportion of 15% data.

This cells will classify the type of data, whether it is a training data, test data, or validation data randomly.

In [25]:
data_type = np.zeros((data_cnt,), dtype=np.int8)

for i in range(0, int(data_cnt * 0.70)):
    data_type[i] = 0

for i in range(int(data_cnt * 0.70), int(data_cnt * 0.85 + 1)):
    data_type[i] = 1

for i in range(int(data_cnt * 0.85 + 1), data_cnt):
    data_type[i] = 2

In [26]:
# Shuffle data
np.random.shuffle(data_type)
data_type

array([0, 1, 2, ..., 2, 1, 0], dtype=int8)

In [27]:
data_type_dist = np.bincount(data_type)

print("Data type distribution:", data_type_dist)

Data type distribution: [17166  3679  3678]


In [28]:
training_rating_dist = np.zeros((5,), dtype=np.int32)

idx = 0
with open("datasets/comments_stemmed.ljson", "r") as fr:
    for line in fr:
        data = json.loads(line)

        if data_type[idx] == 0:
            training_rating_dist[data["rating"] - 1] += 1
        
        idx += 1

In [29]:
print("Training rating distribution:", training_rating_dist)

Training rating distribution: [2313 3056 3779 4048 3970]


### Scheme 1

In this scheme, we will label data training with this rules:
1. Star 5 will be labelled as `[1, 0, 0]`
2. Star 4 will be labelled as `[0.55, 0.45, 0]`
3. Star 3 will be labelled as `[0, 1, 0]`
4. Star 2 will be labelled as `[0, 0.45, 0.55]`
5. Star 1 will be labelled as `[0, 0, 1]`

For testing, the classification is based on this rules:
1. Star 5 and 4 will be labelled as `positive`
2. Star 3 will be labelled as `neutral`
3. Star 2 and 1 will be labelled as `negative`

Those labels are based on the highest element value in resulted vector.

In [30]:
# Calclate data weight
data_weight = np.ones((5,), dtype=np.float64) * np.max(training_rating_dist)
data_weight = data_weight / training_rating_dist
data_weight

array([1.75010808, 1.32460733, 1.07118285, 1.        , 1.01964736])

In [31]:
idx = 0
with open("datasets/comments_stemmed.ljson", "r") as fr:
    with open("datasets/comments_labelled_s1_train.ljson", "w") as ftrain, \
         open("datasets/comments_labelled_s1_val.ljson", "w") as fval, \
         open("datasets/comments_labelled_s1_test.ljson", "w") as ftest:
        for line in fr:
            data = json.loads(line)
            y = []

            if data["rating"] == 1:
                y = [0, 0, 1]
            elif data["rating"] == 2:
                y = [0, 0.45, 0.55]
            elif data["rating"] == 3:
                y = [0, 1, 0]
            elif data["rating"] == 4:
                y = [0.55, 0.45, 0]
            elif data["rating"] == 5:
                y = [1, 0, 0]
                        
            write_data = {
                "X": " ".join(data["text"]),
                "y": y,
                "w": data_weight[data["rating"] - 1]
            }

            write_data = json.dumps(write_data)

            if data_type[idx] == 0:
                ftrain.write(write_data)
                ftrain.write("\n")
            elif data_type[idx] == 1:
                fval.write(write_data)
                fval.write("\n")
            elif data_type[idx] == 2:
                ftest.write(write_data)
                ftest.write("\n")
            
            idx += 1

### Scheme 2

In this scheme, we will label data training with this rules:
1. Star 5 will be labelled as `[1, 0, 0]`
2. Star 4 will be labelled as `[1, 0, 0]`
3. Star 3 will be labelled as `[0, 1, 0]`
4. Star 2 will be labelled as `[0, 0, 1]`
5. Star 1 will be labelled as `[0, 0, 1]`

For testing, the classification is based on this rules:
1. Star 5 and 4 will be labelled as `positive`
2. Star 3 will be labelled as `neutral`
3. Star 2 and 1 will be labelled as `negative`

Those labels are based on the highest element value in resulted vector.

In [32]:
# Calculate data weight
high_dist = training_rating_dist[0] + training_rating_dist[1]
low_dist = training_rating_dist[3] + training_rating_dist[4]
center = training_rating_dist[2]

max_dist = max(high_dist, low_dist, center)

data_weight = np.ones((5,), dtype=np.float64) * max_dist
data_weight = data_weight / np.array([high_dist, high_dist, center, low_dist, low_dist])
data_weight

array([1.49338797, 1.49338797, 2.12172532, 1.        , 1.        ])

In [33]:
idx = 0
with open("datasets/comments_stemmed.ljson", "r") as fr:
    with open("datasets/comments_labelled_s2_train.ljson", "w") as ftrain, \
         open("datasets/comments_labelled_s2_val.ljson", "w") as fval, \
         open("datasets/comments_labelled_s2_test.ljson", "w") as ftest:
        for line in fr:
            data = json.loads(line)
            y = []

            if data["rating"] == 1:
                y = [0, 0, 1]
            elif data["rating"] == 2:
                y = [0, 0, 1]
            elif data["rating"] == 3:
                y = [0, 1, 0]
            elif data["rating"] == 4:
                y = [1, 0, 0]
            elif data["rating"] == 5:
                y = [1, 0, 0]
                        
            write_data = {
                "X": " ".join(data["text"]),
                "y": y,
                "w": data_weight[data["rating"] - 1]
            }

            write_data = json.dumps(write_data)

            if data_type[idx] == 0:
                ftrain.write(write_data)
                ftrain.write("\n")
            elif data_type[idx] == 1:
                fval.write(write_data)
                fval.write("\n")
            elif data_type[idx] == 2:
                ftest.write(write_data)
                ftest.write("\n")
            
            idx += 1

## Classification Model Training

In this section, we will explore about training model for sentiment analysis. We will use 3 hyperparameters with each 2 options. The parameters are below:
1. Word Model:
   We will use `word2vec_model` and `fasttext_model` based on model that we have trained before.
2. Classification Model:
   We will use two different layer type, that is `GRU` and `Bidirectional GRU`. This model structure will be explained in next cell. 
3. Dataset Scheme:
   We will use two scheme that we have build before, that is `Sheme 1` and `Scheme 2`

Below cell is helper function to do our task:

In [34]:
class EmbedGenerator:
    def __init__(self, filename, word_model):
        self.filename = filename
        self.word_model = word_model

    def generate(self):
        with open(self.filename, "r") as f:
            for line in f:
                data = json.loads(line)
                X = []

                for words in data["X"].split():
                    try:
                        X.append(self.word_model.wv.get_vector(words))
                    except KeyError:
                        pass
                
                if len(X) == 0:
                    continue

                X = np.array(X)
                X = X.reshape((1, X.shape[0], X.shape[1]))
                
                y = np.array([data["y"]])
                w = np.array([data["w"]])
                
                yield X, y, w

def train_model(*, word, classification, train_dataset_filename, validation_dataset_filename, name):
    train_gen = EmbedGenerator(train_dataset_filename, word)
    validation_gen = EmbedGenerator(validation_dataset_filename, word)

    train_ds = tf.data.Dataset.from_generator(train_gen.generate, output_signature=(
        tf.TensorSpec(shape=(None, None, INPUT_SIZE), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None,), dtype=tf.float64),
    ))
    validation_ds = tf.data.Dataset.from_generator(validation_gen.generate, output_signature=(
        tf.TensorSpec(shape=(None, None, INPUT_SIZE), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None,), dtype=tf.float64),
    ))

    callback = [
      keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=5,
        restore_best_weights=True,
        min_delta=0.005,
      ),
      keras.callbacks.ModelCheckpoint(
        filepath=f"models/checkpoint/{name}_model_checkpoint.keras",
        save_best_only=True,
      ),
      keras.callbacks.TensorBoard(
        log_dir=f"logs/{name}_{datetime.now().strftime('%Y%m%d-%H%M%S')}",
      ),
    ]

    if classification == "layer_1":
      model = keras.models.Sequential([
          keras.layers.Input(shape=(None, INPUT_SIZE)),
          keras.layers.GRU(128, dropout=0.2),
          keras.layers.Dense(128, activation="relu"),
          keras.layers.Dropout(0.2),
          keras.layers.Dense(128, activation="relu"),
          keras.layers.Dropout(0.2),
          keras.layers.Dense(3, activation="softmax"),
        ]
      )
    elif classification == "layer_2":
      model = keras.models.Sequential([
          keras.layers.Input(shape=(None, INPUT_SIZE)),
          keras.layers.Bidirectional(
            keras.layers.GRU(128, dropout=0.2),
          ),
          keras.layers.Dense(128, activation="relu"),
          keras.layers.Dropout(0.2),
          keras.layers.Dense(128, activation="relu"),
          keras.layers.Dropout(0.2),
          keras.layers.Dense(3, activation="softmax"),
        ]
      )
    else:
      raise ValueError("Invalid classification")
    
    model.compile(
      loss=losses.CategoricalCrossentropy(),
      optimizer=optimizers.Adam(),
      metrics=[keras.metrics.CategoricalAccuracy()],
    )

    history = model.fit(
      train_ds,
      epochs=LEARNING_EPOCH,
      callbacks=callback,
      validation_data=validation_ds,
      batch_size=64,
    )

    model.save(f"models/{name}_model.keras")

    return model, history

In [35]:
word_model = [
  ("word2vec", Word2Vec.load("models/corpus_word2vec.model")),
  ("fasttext", FastText.load("models/corpus_fasttext.model")),
]
classification_scheme = [
  ("layer1", "layer_1"), 
  ("layer2", "layer_2")
]
dataset = [
  ("schema2","datasets/comments_labelled_s2_train.ljson", "datasets/comments_labelled_s2_val.ljson", "datasets/comments_labelled_s2_test.ljson"),
  ("schema1","datasets/comments_labelled_s1_train.ljson", "datasets/comments_labelled_s1_val.ljson", "datasets/comments_labelled_s1_test.ljson"),
]

In [36]:
for dname, train, validation, test in dataset:
    for wname, wmodel in word_model:
        for cname, classification in classification_scheme:
            print(f"Training {dname}_{wname}_{cname} model")
            train_model(
                word=wmodel,
                classification=classification,
                train_dataset_filename=train,
                validation_dataset_filename=validation,
                name=f"{dname}_{wname}_{cname}",
            )
            print()

Training schema2_word2vec_layer1 model


2024-07-21 22:51:40.722213: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-21 22:51:40.753969: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-21 22:51:40.757760: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Epoch 1/100
  17157/Unknown 947s 55ms/step - categorical_accuracy: 0.3521 - loss: 1.5797

2024-07-21 23:07:28.664927: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-07-21 23:07:28.664990: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2024-07-21 23:07:28.665012: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 5211926725495867385
/home/miawheker/.pyenv/versions/3.11.9/lib/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


17160/17160 ━━━━━━━━━━━━━━━━━━━━ 1015s 59ms/step - categorical_accuracy: 0.3521 - loss: 1.5797 - val_categorical_accuracy: 0.3731 - val_loss: 1.2851
Epoch 2/100


...

2024-07-22 05:22:51.886373: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-07-22 05:22:51.886431: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]


17160/17160 ━━━━━━━━━━━━━━━━━━━━ 1878s 109ms/step - categorical_accuracy: 0.5950 - loss: 1.1547 - val_categorical_accuracy: 0.4098 - val_loss: 1.2671
Epoch 6/100


2024-07-22 05:24:21.374966: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-07-22 05:24:21.375016: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]


11128/17160 ━━━━━━━━━━━━━━━━━━━━ 10:23 103ms/step - categorical_accuracy: 0.6140 - loss: 1.1437

KeyboardInterrupt: 